# Perbandingan Kecepatan Enkripsi Algoritma DES, TripleDES, AES, Blowfish

In [7]:
!pip install pycryptodome

In [5]:
from Crypto.Util.Padding import pad, unpad
from Crypto.Random import get_random_bytes
from Crypto.Cipher import DES, DES3, AES, Blowfish
import time

## 1. Inisiasi dan Persiapan

In [12]:
# --- Konfigurasi Uji ---
NUM_ITERATIONS = 10
DATA_SIZES_MB = [0.5, 1, 5, 10, 20]

# Pemetaan Algoritma ke Ukuran Kunci (Bytes)
algorithm_map = {
    'DES': (DES, 8),
    '3DES': (DES3, 24), # 168-bit
    'AES-256': (AES, 32), # 256-bit
    'Blowfish-128': (Blowfish, 16)
}

## 2. Membuat Fungsi untuk mengukur waktu Enkripsi dan Dekripsi

In [15]:
def measure_time(cipher_module, key_size, data_block, is_encryption=True, iterations=10):
    total_time = 0

    # 1. Padding Data: Data harus di-padding sebelum enkripsi
    padded_data = pad(data_block, cipher_module.block_size)

    # 2. Generasi Kunci: Hasilkan kunci unik
    if cipher_module == DES3:
        key = DES3.adjust_key_parity(get_random_bytes(key_size))
    else:
        key = get_random_bytes(key_size)

    for _ in range(iterations):
        # 3. IV: Hasilkan IV baru untuk setiap iterasi (penting untuk MODE_CBC)
        iv = get_random_bytes(cipher_module.block_size)

        # Inisialisasi Cipher
        cipher = cipher_module.new(key, cipher_module.MODE_CBC, iv=iv)

        # Enkripsi
        start_time = time.time()
        if is_encryption:
            ciphertext = cipher.encrypt(padded_data)
        else: # Dekripsi (membutuhkan hasil ciphertext dari enkripsi sebelumnya)
             # Catatan: Dalam pengujian nyata, Anda akan menguji dekripsi pada ciphertext yang sudah dienkripsi.
             # Untuk kesederhanaan, asumsikan ciphertext sudah tersedia.
             cipher.decrypt(padded_data) # Menggunakan padded_data sebagai placeholder ciphertext
        end_time = time.time()

        total_time += (end_time - start_time)

    return total_time / iterations # Waktu rata-rata

## 3. Menghitung Waktu rata-rata dan throughput

In [16]:
results = []

print("Memulai Simulasi Kinerja (Enkripsi)...")

for algo_name, (module, key_size) in algorithm_map.items():
    print(f"\n--- Menguji {algo_name} ---")

    for size_mb in DATA_SIZES_MB:
        # 1. Siapkan blok data acak untuk ukuran saat ini
        data_block = get_random_bytes(int(size_mb * 1024 * 1024))

        # 2. Ukur waktu rata-rata enkripsi
        avg_enc_time = measure_time(
            module,
            key_size,
            data_block,
            is_encryption=True,
            iterations=NUM_ITERATIONS
        )

        # 3. Hitung Throughput
        throughput = size_mb / avg_enc_time

        # 4. Simpan hasil
        results.append({
            'Algoritma': algo_name,
            'Ukuran Data (MB)': size_mb,
            'Waktu Rata-Rata (s)': avg_enc_time,
            'Throughput (MB/s)': throughput
        })
        print(f"  {size_mb} MB: {avg_enc_time:.4f} s | {throughput:.2f} MB/s")

Memulai Simulasi Kinerja (Enkripsi)...

--- Menguji DES ---
  0.5 MB: 0.0096 s | 51.99 MB/s
  1 MB: 0.0185 s | 54.18 MB/s
  5 MB: 0.0935 s | 53.49 MB/s
  10 MB: 0.1641 s | 60.95 MB/s
  20 MB: 0.3166 s | 63.17 MB/s

--- Menguji 3DES ---
  0.5 MB: 0.0226 s | 22.16 MB/s
  1 MB: 0.0418 s | 23.93 MB/s
  5 MB: 0.2171 s | 23.03 MB/s
  10 MB: 0.4498 s | 22.23 MB/s
  20 MB: 0.8639 s | 23.15 MB/s

--- Menguji AES-256 ---
  0.5 MB: 0.0012 s | 415.06 MB/s
  1 MB: 0.0025 s | 396.65 MB/s
  5 MB: 0.0133 s | 377.29 MB/s
  10 MB: 0.0277 s | 361.61 MB/s
  20 MB: 0.0568 s | 352.11 MB/s

--- Menguji Blowfish-128 ---
  0.5 MB: 0.0049 s | 101.06 MB/s
  1 MB: 0.0100 s | 100.32 MB/s
  5 MB: 0.0525 s | 95.23 MB/s
  10 MB: 0.1076 s | 92.91 MB/s
  20 MB: 0.2117 s | 94.46 MB/s
